<a href="https://colab.research.google.com/github/anshupandey/AI_Agents/blob/main/AAP_C3_gemini_parallel_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Parallel Function Calls and Multiple Function Responses in Gemini

## Overview

Gemini is a family of generative AI models developed by Google DeepMind that is designed for multimodal use cases. The Gemini API gives you access to the Gemini Pro and Gemini Pro Vision models.

[Function Calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) in Gemini lets you create a description of a function in your code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with.

In this tutorial, you'll learn how to work with parallel function calling within Gemini Function Calling, including:
    
- Handling parallel function calls for repeated functions
- Working with parallel function calls across multiple functions
- Extracting multiple function calls from a Gemini response
- Calling multiple functions and returning them to Gemini

### What is parallel function calling?

In previous versions of the Gemini Pro models (prior to May 2024), Gemini would return two or more chained function calls if the model determined that more than one function call was needed before returning a natural language summary. Here, a chained function call means that you get the first function call response, return the API data to Gemini, get a second function call response, return the API data to Gemini, and so on.

In recent versions of specific Gemini Pro models (from May 2024 and on), Gemini has the ability to return two or more function calls in parallel (i.e., two or more function call responses within the first function call response object). Parallel function calling allows you to fan out and parallelize your API calls or other actions that you perform in your application code, so you don't have to work through each function call response and return one-by-one! Refer to the [Gemini Function Calling documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling) for more information on which Gemini model versions support parallel function calling.


<img src="https://storage.googleapis.com/github-repo/generative-ai/gemini/function-calling/parallel-function-calling-in-gemini.png">

## Getting Started


### Install Vertex AI SDK and other required packages

In [ ]:
!pip3 install --upgrade --user --quiet google-cloud-aiplatform wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>



### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "jrproject-402905"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Code Examples

### Import libraries

In [ ]:
import wikipedia
from IPython.display import display, Markdown
from typing import List, Dict
from vertexai.generative_models import (
    FunctionDeclaration,
    GenerativeModel,
    GenerationConfig,
    GenerationResponse,
    Part,
    Tool,
)

### Define helper function

In [ ]:
# Helper function to extract one or more function calls from a Gemini Function Call response
def extract_function_calls(response: GenerationResponse) -> List[Dict]:
    function_calls = []
    if response.candidates[0].function_calls:
        for function_call in response.candidates[0].function_calls:
            function_call_dict = {function_call.name: {}}
            for key, value in function_call.args.items():
                function_call_dict[function_call.name][key] = value
            function_calls.append(function_call_dict)
    return function_calls

## Example: Parallel function calls on the same function

A great use case for parallel function calling is when you have a function that only accepts one parameter per API call and you need to make repeated calls to that function.

With Parallel Function Calling, rather than having to send N number of API requests to Gemini for N number function calls, instead you can send a single API request to Gemini, receive N number of Function Call Responses within a single response, make N number of external API calls in your code, then return all of the API responses to Gemini in bulk. And you can do all of this without any extra configuration in your function declarations, tools, or requests to Gemini.

In this example, you'll do exactly that and use Parallel Function Calling in Gemini to ask about multiple topics on [Wikipedia](https://www.wikipedia.org/). Let's get started!

### Write function declarations and wrap them in a tool

First, define your function declarations and tool using the Vertex AI Python SDK:

In [ ]:
search_wikipedia = FunctionDeclaration(
    name="search_wikipedia",
    description="Search for articles on Wikipedia",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Query to search for on Wikipedia",
            },
        },
    },
)

wikipedia_tool = Tool(
    function_declarations=[
        search_wikipedia,
    ],
)

### Initialize the Gemini model

Now you can initialize Gemini using a [model version that supports parallel function calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling):

In [ ]:
model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[wikipedia_tool],
)
chat = model.start_chat()

### Send prompt to Gemini

Send a prompt to Gemini that includes a phrase that you expect to invoke two or more function calls.

In this case we'll ask about three different article topics to search for on Wikipedia:

In [ ]:
prompt = "Search for articles related to solar panels, renewable energy, and battery storage and provide a summary of your findings"

response = chat.send_message(prompt)

### Extract function names and parameters

Use the helper function that we created earlier to extract the function names and function parameters for each Function Call that Gemini responded with:

In [ ]:
function_calls = extract_function_calls(response)
function_calls

[{'search_wikipedia': {'query': 'Solar panels'}},
 {'search_wikipedia': {'query': 'Renewable energy'}},
 {'search_wikipedia': {'query': 'Battery storage'}}]

Note that the helper function is just one way to extract fields from the structured Function Call response. You can modify the helper function or write your own custom code to extract and get the fields into your preferred format or data structure!

### Make external API calls

Next, you'll loop through the Function Calls and use the `wikipedia` Python package to make an API call for each search query:

In [ ]:
api_response = []

# Loop over multiple function calls
for function_call in function_calls:
    print(function_call)

    # Make external API call
    result = wikipedia.summary(function_call["search_wikipedia"]["query"])

    # Collect all API responses
    api_response.append(result)

{'search_wikipedia': {'query': 'Solar panels'}}
{'search_wikipedia': {'query': 'Renewable energy'}}
{'search_wikipedia': {'query': 'Battery storage'}}


### Get a natural language summary

Now you can return all of the API responses to Gemini so that it can generate a natural language summary:

In [ ]:
# Return the API response to Gemini

function_responses = []
for i in range(len(function_calls)):
    function_responses.append(
        Part.from_function_response(
            name="search_wikipedia",
            response={
                "content": api_response[i],
            },
        )
    )

response = chat.send_message(function_responses)
display(Markdown(response.text))

Solar panels convert sunlight into electricity using photovoltaic cells, offering a renewable and clean energy source that can reduce greenhouse gas emissions and lower electricity bills. However, they depend on sunlight availability, require cleaning, and have high initial costs. 

Renewable energy, including solar, wind, and hydropower, is replenished naturally and has become increasingly efficient and cheaper. It's vital for combating climate change by reducing greenhouse gas emissions compared to fossil fuels. While facing obstacles like fossil fuel subsidies and land use concerns, the transition to renewables is crucial for a sustainable future.

Battery storage power stations are essential for stabilizing electric grids by storing energy from sources like solar and wind power. They provide rapid response to grid fluctuations, ensuring reliability. While their capacity is smaller than pumped-storage hydropower, battery storage is rapidly growing and becoming more cost-effective, making it a key player in the transition to a cleaner and more resilient energy system. 


And you're done with no extra configuration necessary!

Note that Gemini will use the information in your `FunctionDeclarations` to determine if and when it should return parallel Function Call responses, or it will determine which Function Calls need to be made before others to get information that a subsequent Function Call depends on. So be sure to account for this behavior in your logic and application code!

## Example: Parallel function calls across multiple functions

Another good fit for parallel function calling is when you have multiple, independent functions that you want to be able to call in parallel, which reduces the number of consecutive Gemini API calls that you need to make and (ideally) reduces the overall response time to the end user who is waiting for a natural language response.

In this example, you'll use Parallel Function Calling in Gemini to ask about multiple aspects of topics and articles on [Wikipedia](https://www.wikipedia.org/).

### Write function declarations and wrap them in a tool

First, define your function declarations and tool using the Vertex AI Python SDK:

In [ ]:
search_wikipedia = FunctionDeclaration(
    name="search_wikipedia",
    description="Search for articles on Wikipedia",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Query to search for on Wikipedia",
            },
        },
    },
)

suggest_wikipedia = FunctionDeclaration(
    name="suggest_wikipedia",
    description="Get suggested titles from Wikipedia for a given term",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Query to search for suggested titles on Wikipedia",
            },
        },
    },
)

summarize_wikipedia = FunctionDeclaration(
    name="summarize_wikipedia",
    description="Get article summaries from Wikipedia",
    parameters={
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "Query to search for article summaries on Wikipedia",
            },
        },
    },
)

wikipedia_tool = Tool(
    function_declarations=[
        search_wikipedia,
        suggest_wikipedia,
        summarize_wikipedia,
    ],
)

### Initialize the Gemini model

Now you can initialize Gemini using a [model version that supports parallel function calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling):

In [ ]:
model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[wikipedia_tool],
)
chat = model.start_chat()

### Send prompt to Gemini

Send a prompt to Gemini that includes a phrase that you expect to invoke two or more function calls.

In this case we'll ask about three types of details to look up for a given topic on Wikipedia:

In [ ]:
prompt = "Show the search results, variations, and article summaries about Wikipedia articles related to the solar system"

response = chat.send_message(prompt)

### Extract function names and parameters

Use the helper function that we created earlier to extract the function names and function parameters for each Function Call that Gemini responded with:

In [ ]:
function_calls = extract_function_calls(response)
function_calls

[{'search_wikipedia': {'query': 'solar system'}},
 {'suggest_wikipedia': {'query': 'solar system'}},
 {'summarize_wikipedia': {'topic': 'solar system'}}]

### Make external API calls

Next, you'll loop through the Function Calls and use the `wikipedia` Python package to make APIs calls and gather information from Wikipedia:

In [ ]:
api_response = {}

# Loop over multiple function calls
for function_call in function_calls:
    # Extract the function name
    print(function_call)
    for key in function_call:
        function_name = key

    # Determine which external API call to make
    if function_name == "search_wikipedia":
        result = wikipedia.search(function_call["search_wikipedia"]["query"])
    if function_name == "suggest_wikipedia":
        result = wikipedia.suggest(function_call["suggest_wikipedia"]["query"])
    if function_name == "summarize_wikipedia":
        result = wikipedia.summary(
            function_call["summarize_wikipedia"]["topic"], auto_suggest=False
        )

    # Collect all API responses
    api_response[function_name] = result

{'search_wikipedia': {'query': 'solar system'}}
{'suggest_wikipedia': {'query': 'solar system'}}
{'summarize_wikipedia': {'topic': 'solar system'}}


### Get a natural language summary

Now you can return all of the API responses to Gemini so that it can generate a natural language summary:

In [ ]:
# Return the API response to Gemini
function_response = []
for function_name in api_response:
    function_response.append(
        Part.from_function_response(
            name=function_name,
            response={
                "content": api_response[function_name],
            },
        )
    )

response = chat.send_message(function_response)

display(Markdown(response.text))

Here's some information about the Solar System:

Search results: "Solar System", "Formation and evolution of the Solar System", "Photovoltaic system", "Solar System (disambiguation)", "List of Solar System objects by size", "Small Solar System body", "List of Solar System objects", "Solar System belts", "Passive solar building design", "List of natural satellites"

Variations: "soler system"

Summary: The Solar System, formed around 4.6 billion years ago, consists of the Sun and objects orbiting it. The Sun, a G-type main-sequence star, generates energy through hydrogen fusion. Eight planets orbit the Sun: Mercury, Venus, Earth, Mars (terrestrial planets), Jupiter, Saturn (gas giants), Uranus, and Neptune (ice giants). The Solar System also contains dwarf planets (Ceres, Pluto, Eris, etc.), asteroids, comets, and more.  The Sun's influence, the solar wind, extends far beyond the planets, forming the heliosphere. The Oort cloud, a distant region, marks the Solar System's edge. Proxima Centauri, the nearest star, lies 4.25 light-years away. 


And you're done! You successfully made parallel function calls for a couple of different use cases. Feel free to adapt the code samples here for your own use cases and applications. Or try another notebook to continue exploring other functionality in the Gemini API.

Happy parallel function calling!